In [1]:
Nx = 300
Ny = 300
NxNy = Nx * Ny

dx = 0.03
dy = 0.03

0.03

Time integration parameters

In [2]:
nstep = 4000
nprint = 50;

In [3]:
dtime = 1.e-4

0.0001

Material specific parameters


In [4]:
tau = 0.0003
epsilonb = 0.01
kappa = 1.8
delta = 0.02
aniso = 4.0
alpha = 0.9
teq = 1.0
theta0 = 0.2
seed = 5.0

pix = 4.0 * atan(1.0)

3.141592653589793

Initialize and introduce the initial nuclei:

In [ ]:
phi, tempr = nucleus(Nx,Ny, seed)

In [ ]:
for istep = 1:nstep
    for i = 1 : Nx
        for j = 1 : Ny
            
            jp = j + 1
            jm = j - 1
            ip = i + 1
            im = i - 1
            
            if (im == 0)
                im = Nx
            end
            if (ip == (Nx + 1))
                ip = 1
            end
            
            if (jp == (Ny + 1))
                jp = 1
            end
            if (jm == 0)
                jm = Ny
            end
            
            hne = phi(ip,j)
            hnw = phi(im,j)
            hns = phi(i,jm)
            hnn = phi(i,jp)
            hnc = phi(i,j)
            
            lap_phi(i,j) = (hnw + hne + hns + hnn -4.0*hnc)/(dx*dx)
            
            hne = tempr(ip,j)
            hnw = tempr(im,j)
            hns = tempr(i,jm)
            hnn = tempr(i,jp)
            hnc = tempr(i,j)
            
            lap_tempr(i,j) = (hnw + hne + hns + hnn -4.0*hnc)/(dx*dx)
            
            phidx(i,j) = (phi(ip,j)-phi(im,j))/dx
            phidy(i,j) = (phi(i,jp)-phi(i,jm))/dy
            
            # calculate angle
            theta = atan2(phidy(i,j), phidx(i,j))
            
            # epsilon and its derivative:
            epsilon[i,j] = -epsilonb * aniso * delta * sin(aniso*(theta-theta0))
            
        end
    end
    
    
    for i = 1 : Nx
        for j = 1 : Ny
            jp = j + 1
            jm = j - 1
            
            ip = i + 1
            im = i - 1
            
            if (im == 0)
                im = Nx
            end
            if (ip == Nx + 1)
                ip = 1
            end
            if (jm == 0)
                jm = Ny
            end
            if (jp == Ny + 1)
                jp = 1
            end
            
            phiold = phi[i,j]
            
            # first term
            term1 = (epsilon[i,jp] * epsilon_deriv[i,jp] * phidx[i,jp] - epsilon[i,jm] * epsilon_deriv[i,jm]*phidx[i,jm])/dy
            # second term
            term2 = -(epsilon[ip,j] * epsilon_deriv[ip,j]*phidy[ip,j] - epsilon[im,j] * epsilon_deriv[im,j]*phidy[im,j])/dy
            
            # factor m
            m = alpha/pi*atan(gamma*(teq-tempr[i,j]))
            
            # time integration
            phi[i,j] = phi[i,j]+(dtime/tau)*(term1 + term2 + epsilon[i,j]^2 * lap_phi[i,j] + 
                       phiold*(1.0-phiold)*(phiold-0.5+m))
            
            # evolve temperature
            tempr[i,j] = tempr[i,j] + dtime *lap_tempr[i,j] + kappa*(phi[i,j]-phiold)
        end
    end
    
    if (istep % nprint == 0)
        @printf("done step %5d\n",istep)
        # write vtk file
        write_vtk_grid_values(Nx,Ny,dx,dy,istep,phi,tempr);
    end  
end
            
            
            